In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

from config.experiment_config import ExperimentConfig

In [3]:
data_df = None
for file in os.listdir(ExperimentConfig.PATH_FEATURES):
    if file not in ("ipos.xlsx", "spx_interim.xlsx"):
        print(file)
        if file not in (
            "bbb_credit.xlsx",
            "global_bonds_total_r.xlsx",
            "10y.xlsx",
            "default_yield.xlsx",
            "inflation_total_r.xlsx",
            "libor3m.xlsx",
            "indust_prod.xlsx",
            "sentiment.xlsx",
        ):
            interim_df = pd.read_excel(
                ExperimentConfig.PATH_FEATURES / file,
                parse_dates=True,
                date_format="%Y-%m-%d",
                skiprows=6,
            )
        else:
            interim_df = pd.read_excel(
                ExperimentConfig.PATH_FEATURES / file,
                parse_dates=True,
                date_format="%Y-%m-%d",
                skiprows=5,
            )

        # print(interim_df.head())
        interim_df = interim_df.rename(
            columns={
                "PX_LAST": f"{file.split('.xlsx')[0]}",
                "Date": "date",
                "PX_BID": f"{file.split('.xlsx')[0]}_bid",
            }
        )
        if "CHG_PCT_1D" in interim_df.columns:
            interim_df = interim_df.drop(columns=["CHG_PCT_1D"])
        interim_df["date"] = pd.to_datetime(interim_df["date"], format="%d.%m.%Y")
        interim_df = interim_df.set_index("date")
        interim_df = interim_df.sort_index()

        if data_df is None:
            data_df = interim_df
        else:
            data_df = pd.merge_asof(
                data_df,
                interim_df,
                left_index=True,
                right_index=True,
                tolerance=pd.Timedelta(days=1),
                direction="backward",
            )
data_df

spx.xlsx
ishares_pure_credit.xlsx
world_etf.xlsx
bbb_credit.xlsx
global_bonds_total_r.xlsx
si.xlsx
10y.xlsx
default_yield.xlsx
spx_add.xlsx
tail.xlsx
est_pe.xlsx
erp.xlsx
ts.xlsx
inflation_swap.xlsx
inflation_total_r.xlsx
libor3m.xlsx
indust_prod.xlsx
sentiment.xlsx


,spx,spx_bid,DVD_PAYOUT_RATIO,DVD_SH_LAST,NET_AGGTE_DVD_YLD,PE_RATIO,BEST_CAPEX,PX_TO_BOOK_RATIO,NET_DEBT_TO_EBITDA,ishares_pure_credit,...,est_pe,BEST_EPS,erp,erp_bid,ts,inflation_swap,inflation_total_r,libor3m,indust_prod,sentiment
date,,,,,,,,,,,,,,,,,,,,,
1950-01-30,17.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-01-31,17.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-02-01,17.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-02-02,17.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-02-03,17.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-22,6086.37,6017.21,37.2874,0.104107,1.23,27.3965,-120.6609,5.2880,1.4806,24.5550,...,6086.37,238.3995,293.42,NaN,30.919,2.5397,323.3708,NaN,NaN,NaN
2025-01-23,6118.71,6052.69,37.2874,0.098825,1.23,27.5421,-120.7156,5.3161,1.4806,24.5600,...,6118.71,238.2551,296.42,NaN,35.033,2.5770,323.4737,NaN,NaN,43.4
2025-01-24,6101.24,6055.63,37.2874,0.572570,1.23,27.4634,-120.2167,5.3009,1.4806,24.5700,...,6101.24,238.0381,294.73,NaN,34.959,2.5557,325.4554,NaN,NaN,43.4


In [4]:
data_df = data_df.dropna(subset=["spx"])
data_df["spread"] = data_df["spx"] - data_df["spx_bid"]
data_df["spx"] = data_df["spx"] / data_df["spx"].shift(1) - 1
data_df

/var/folders/sr/lzvmv9j54ks_jl4x99t250r00000gn/T/ipykernel_97970/100820549.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["spread"] = data_df["spx"] - data_df["spx_bid"]
/var/folders/sr/lzvmv9j54ks_jl4x99t250r00000gn/T/ipykernel_97970/100820549.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["spx"] = data_df["spx"] / data_df["spx"].shift(1) - 1


,spx,spx_bid,DVD_PAYOUT_RATIO,DVD_SH_LAST,NET_AGGTE_DVD_YLD,PE_RATIO,BEST_CAPEX,PX_TO_BOOK_RATIO,NET_DEBT_TO_EBITDA,ishares_pure_credit,...,BEST_EPS,erp,erp_bid,ts,inflation_swap,inflation_total_r,libor3m,indust_prod,sentiment,spread
date,,,,,,,,,,,,,,,,,,,,,
1950-01-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-01-31,0.001763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-02-01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-02-02,0.010557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-02-03,0.003482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-22,0.006138,6017.21,37.2874,0.104107,1.23,27.3965,-120.6609,5.2880,1.4806,24.5550,...,238.3995,293.42,NaN,30.919,2.5397,323.3708,NaN,NaN,NaN,69.16
2025-01-23,0.005314,6052.69,37.2874,0.098825,1.23,27.5421,-120.7156,5.3161,1.4806,24.5600,...,238.2551,296.42,NaN,35.033,2.5770,323.4737,NaN,NaN,43.4,66.02
2025-01-24,-0.002855,6055.63,37.2874,0.572570,1.23,27.4634,-120.2167,5.3009,1.4806,24.5700,...,238.0381,294.73,NaN,34.959,2.5557,325.4554,NaN,NaN,43.4,45.61


In [5]:
data_df["rate"] = data_df["10y"] / 100 / 365
data_df["day_count"] = data_df.index.diff().days
data_df.iloc[0, 1] = 1
data_df["acc_rate"] = data_df["rate"] * data_df["day_count"]

/var/folders/sr/lzvmv9j54ks_jl4x99t250r00000gn/T/ipykernel_97970/1023105569.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["rate"] = data_df["10y"] / 100 / 365
/var/folders/sr/lzvmv9j54ks_jl4x99t250r00000gn/T/ipykernel_97970/1023105569.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["day_count"] = data_df.index.diff().days
/var/folders/sr/lzvmv9j54ks_jl4x99t250r00000gn/T/ipykernel_97970/1023105569.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [6]:
data_df

,spx,spx_bid,DVD_PAYOUT_RATIO,DVD_SH_LAST,NET_AGGTE_DVD_YLD,PE_RATIO,BEST_CAPEX,PX_TO_BOOK_RATIO,NET_DEBT_TO_EBITDA,ishares_pure_credit,...,ts,inflation_swap,inflation_total_r,libor3m,indust_prod,sentiment,spread,rate,day_count,acc_rate
date,,,,,,,,,,,,,,,,,,,,,
1950-01-30,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-01-31,0.001763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1950-02-01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1950-02-02,0.010557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1950-02-03,0.003482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-22,0.006138,6017.21,37.2874,0.104107,1.23,27.3965,-120.6609,5.2880,1.4806,24.5550,...,30.919,2.5397,323.3708,NaN,NaN,NaN,69.16,0.000126,1.0,0.000126
2025-01-23,0.005314,6052.69,37.2874,0.098825,1.23,27.5421,-120.7156,5.3161,1.4806,24.5600,...,35.033,2.5770,323.4737,NaN,NaN,43.4,66.02,0.000127,1.0,0.000127
2025-01-24,-0.002855,6055.63,37.2874,0.572570,1.23,27.4634,-120.2167,5.3009,1.4806,24.5700,...,34.959,2.5557,325.4554,NaN,NaN,43.4,45.61,0.000127,1.0,0.000127


In [7]:
data_df = data_df.drop(["spx_bid"], axis=1)
data_df = data_df.ffill()
# data_df = data_df.dropna(axis=1)
data_df

,spx,DVD_PAYOUT_RATIO,DVD_SH_LAST,NET_AGGTE_DVD_YLD,PE_RATIO,BEST_CAPEX,PX_TO_BOOK_RATIO,NET_DEBT_TO_EBITDA,ishares_pure_credit,ishares_pure_credit_bid,...,ts,inflation_swap,inflation_total_r,libor3m,indust_prod,sentiment,spread,rate,day_count,acc_rate
date,,,,,,,,,,,,,,,,,,,,,
1950-01-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-01-31,0.001763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1950-02-01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1950-02-02,0.010557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1950-02-03,0.003482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-22,0.006138,37.2874,0.104107,1.23,27.3965,-120.6609,5.2880,1.4806,24.5550,24.55,...,30.919,2.5397,323.3708,4.85372,0.92,25.4,69.16,0.000126,1.0,0.000126
2025-01-23,0.005314,37.2874,0.098825,1.23,27.5421,-120.7156,5.3161,1.4806,24.5600,24.56,...,35.033,2.5770,323.4737,4.85372,0.92,43.4,66.02,0.000127,1.0,0.000127
2025-01-24,-0.002855,37.2874,0.572570,1.23,27.4634,-120.2167,5.3009,1.4806,24.5700,24.55,...,34.959,2.5557,325.4554,4.85372,0.92,43.4,45.61,0.000127,1.0,0.000127


In [8]:
features = []
FIRST_VALID_DATE = pd.Timestamp("2004-03-19")
for column in data_df.columns:
    first_valid_index = data_df[column].first_valid_index()
    print(f"{column} -> {first_valid_index}")
    if first_valid_index is not None and first_valid_index <= FIRST_VALID_DATE:
        features.append(column)

spx -> 1950-01-31 00:00:00
DVD_PAYOUT_RATIO -> 1991-02-04 00:00:00
DVD_SH_LAST -> 1980-02-29 00:00:00
NET_AGGTE_DVD_YLD -> 1990-01-31 00:00:00
PE_RATIO -> 1990-05-10 00:00:00
BEST_CAPEX -> 2007-12-18 00:00:00
PX_TO_BOOK_RATIO -> 1990-05-10 00:00:00
NET_DEBT_TO_EBITDA -> 1991-02-04 00:00:00
ishares_pure_credit -> 2015-09-11 00:00:00
ishares_pure_credit_bid -> 2015-09-11 00:00:00
world_etf -> 2012-01-12 00:00:00
world_etf_bid -> 2012-02-21 00:00:00
bbb_credit -> 1986-01-02 00:00:00
global_bonds_total_r -> 1990-01-02 00:00:00
si -> 2013-02-05 00:00:00
si_bid -> 2020-06-30 00:00:00
10y -> 1962-01-02 00:00:00
default_yield -> 1987-01-30 00:00:00
spx_add -> 1950-01-30 00:00:00
spx_add_bid -> 1998-03-02 00:00:00
CORR_COEF -> 1950-01-30 00:00:00
SHORT_AND_LONG_TERM_DEBT -> 1990-05-10 00:00:00
tail -> 2004-03-19 00:00:00
tail_bid -> None
est_pe -> 1979-02-16 00:00:00
BEST_EPS -> 1990-03-30 00:00:00
erp -> 1997-09-09 00:00:00
erp_bid -> 2023-01-18 00:00:00
ts -> 1977-01-31 00:00:00
inflation_swa

In [9]:
data_df = data_df[features]
data_df = data_df.loc[FIRST_VALID_DATE:]
data_df

,spx,DVD_PAYOUT_RATIO,DVD_SH_LAST,NET_AGGTE_DVD_YLD,PE_RATIO,PX_TO_BOOK_RATIO,NET_DEBT_TO_EBITDA,bbb_credit,global_bonds_total_r,10y,...,erp,ts,inflation_total_r,libor3m,indust_prod,sentiment,spread,rate,day_count,acc_rate
date,,,,,,,,,,,,,,,,,,,,,
2004-03-19,-0.011173,37.4035,0.004168,1.61,19.9807,3.1030,4.6535,232.90,298.3774,3.7710,...,78.67,225.576,190.9748,1.11000,0.62,38.60,0.22,0.000103,1.0,0.000103
2004-03-22,-0.012958,37.4035,0.015004,1.63,19.7218,3.0628,4.6535,234.74,299.2598,3.7126,...,77.14,223.155,191.9312,1.11000,0.62,38.60,0.49,0.000102,3.0,0.000305
2004-03-23,-0.001324,37.4035,0.000000,1.63,19.6957,3.0587,4.6535,236.99,298.7925,3.6901,...,76.92,222.618,191.7853,1.11000,0.62,38.60,0.13,0.000101,1.0,0.000101
2004-03-24,-0.002395,37.4035,0.004144,1.64,19.6485,3.0514,4.6535,235.31,298.4857,3.7069,...,76.92,224.636,191.4800,1.11000,0.62,38.60,0.27,0.000102,1.0,0.000102
2004-03-25,0.016365,37.3791,0.000000,1.61,19.9759,3.0975,4.6614,235.32,298.1168,3.7368,...,78.21,226.068,190.6442,1.11000,0.62,31.48,0.57,0.000102,1.0,0.000102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-22,0.006138,37.2874,0.104107,1.23,27.3965,5.2880,1.4806,142.72,464.1360,4.6109,...,293.42,30.919,323.3708,4.85372,0.92,25.40,69.16,0.000126,1.0,0.000126
2025-01-23,0.005314,37.2874,0.098825,1.23,27.5421,5.3161,1.4806,140.88,463.2555,4.6436,...,296.42,35.033,323.4737,4.85372,0.92,43.40,66.02,0.000127,1.0,0.000127
2025-01-24,-0.002855,37.2874,0.572570,1.23,27.4634,5.3009,1.4806,145.86,465.5408,4.6214,...,294.73,34.959,325.4554,4.85372,0.92,43.40,45.61,0.000127,1.0,0.000127


In [10]:
data_df.isna().any().any()

np.False_

In [11]:
data_df[data_df.columns.difference(["rate", "day_count"])].to_csv(
    ExperimentConfig.PATH_OUTPUT / ExperimentConfig.INITIAL_FEATURES_FILENAME
)